<a href="https://colab.research.google.com/github/kruthi03/wind-speed/blob/Stats%26ML/100001319_Exercise_4_Kruthika_P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

# Display the first few rows
print(data.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


Handling missing values

In [ ]:
missing_values = df.isnull().sum() # Change 'df' to 'data'
print(missing_values)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


Standardization (Z-score) and Normalization (Min-Max Scaling)

In [ ]:
# Impute Age with the median
df['Age'].fillna(df['Age'].median(), inplace=True)

# Impute Embarked with the mode
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)


<ipython-input-7-af115de9a1b2>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
<ipython-input-7-af115de9a1b2>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])


In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
df[['Age', 'Fare']] = min_max_scaler.fit_transform(df[['Age', 'Fare']])


Handling Noise
Injecting Random Noise and Removing It

In [ ]:
import numpy as np
noise = np.random.normal(0, 1, size=df['Age'].shape)
df['Age'] += noise

In [ ]:
df['Age'] = df['Age'].rolling(window=3, min_periods=1).mean()


In [ ]:
df['Age'] = df['Age'].rolling(window=3, min_periods=1).mean()


Handling Outliers
Detecting Outliers Using Z-score

In [ ]:
from scipy.stats import zscore
df['Age_zscore'] = zscore(df['Age'])
outliers = df[df['Age_zscore'].abs() > 3]
print(outliers)


     PassengerId  Survived  Pclass  \
160          161         0       3   
161          162         1       2   

                                                  Name     Sex       Age  \
160                           Cribb, Mr. John Hatfield    male -1.138499   
161  Watt, Mrs. James (Elizabeth "Bessie" Inglis Mi...  female -1.209618   

     SibSp  Parch      Ticket      Fare Cabin Embarked  Age_zscore  
160      0      1      371362  0.031425   NaN        S   -3.047263  
161      0      0  C.A. 33595  0.030742   NaN        S   -3.191119  


In [ ]:
df = df[df['Age_zscore'].abs() <= 3]


In [ ]:
df['Age'] = np.log1p(df['Age'])


Feature Selection

Correlation Matrix

In [ ]:
# Select only numeric columns before calculating correlation
numeric_df = df.select_dtypes(include=['number'])

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

# Print the correlation with 'Survived'
print(correlation_matrix['Survived'].sort_values(ascending=False))

Survived       1.000000
Fare           0.257845
Parch          0.083125
PassengerId   -0.004428
Age_zscore    -0.028646
Age           -0.032761
SibSp         -0.035120
Pclass        -0.337801
Name: Survived, dtype: float64


Mutual Information

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mi = mutual_info_classif(df.drop('Survived', axis=1), df['Survived'])
print(mi)

[0.01299782 0.07250748 0.00221798 0.16156632 0.         0.00963223
 0.03342491 0.11721202 0.13585057 0.04709909 0.01595091 0.        ]


Wrapper Methods
Recursive Feature Elimination
Backward Elimination


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Select only object (string) columns for encoding
object_cols = df.select_dtypes(include=['object']).columns

# Apply Label Encoding to each object column
for col in object_cols:
    df[col] = le.fit_transform(df[col])

# Create the model and RFE object
model = LogisticRegression()
rfe = RFE(estimator=model, n_features_to_select=5)

# Fit the RFE object to the transformed data
fit = rfe.fit(df.drop('Survived', axis=1), df['Survived'])

# Print the results
print(fit.support_)
print(fit.ranking_)

[False  True False  True  True False False False  True False  True False]
[6 1 7 1 1 2 4 8 1 5 1 3]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(df.drop('Survived', axis=1), df['Survived'])
print(lasso.coef_)


[ 1.43290184e-06 -1.04513666e-01 -6.44892819e-05 -4.73641267e-01
 -0.00000000e+00 -1.98853088e-02 -0.00000000e+00 -1.49233926e-04
  0.00000000e+00 -7.81378408e-04 -2.96687881e-02 -3.90042000e-03]
